In [ ]:
!pip install mlflow


In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os # navigate through defferent files
import numpy as np
import wandb
import mlflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import epsilon

Loading data

In [ ]:
import wandb
import mlflow
import tensorflow as tf

# Initialize Weights & Biases (W&B)
wandb.login()
wandb.init(project="toxicity-detection")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: e20189 (e20189-university-of-peradeniya) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CommentToxicity/jigsaw-toxic-comment-classification-challenge/train.csv/train.csv') # train data set

In [ ]:
df[df['toxic'] ==1].head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\r\n@ | talk .\r\nWhat is i...,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \r\n\r\nDon't look, come or think of comm...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \r\n\r\nArchange...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


In [ ]:
id = 12
df.iloc[id]['comment_text']


'Hey... what is it..\r\n@ | talk .\r\nWhat is it... an exclusive group of some WP TALIBANS...who are good at destroying, self-appointed purist who GANG UP any one who asks them questions abt their ANTI-SOCIAL and DESTRUCTIVE (non)-contribution at WP?\r\n\r\nAsk Sityush to clean up his behavior than issue me nonsensical warnings...'

In [ ]:
df[df.columns[2:]].iloc[id]

,12
toxic,1
severe_toxic,0
obscene,0
threat,0
insult,0
identity_hate,0


Preprocessing

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [ ]:
y = df[df.columns[2:]].values # converting the measuring values in the data frames to a matrix as labels

In [ ]:
x = df['comment_text'] # take the comment text as inputs

In [ ]:
MAX_WORDS = 200000 # Number of vocab in the dictionary
vectorizer = TextVectorization(max_tokens=MAX_WORDS , output_sequence_length=1800 ,output_mode='int') # cap the max length of inputs to 1800 words and the model outputs data in the integer format

In [ ]:
vectorizer.adapt(x.values) # fit the vectorizer to the data , learn the words in the inputs

In [ ]:
import tensorflow as tf
import pickle

# Save vectorizer weights
vectorizer_weights = vectorizer.get_weights()
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer_weights, f)


In [ ]:
import pickle
from tensorflow.keras.layers import TextVectorization

#  Save the vocabulary
vocab = vectorizer.get_vocabulary()
with open("vectorizer_vocab.pkl", "wb") as f:
    pickle.dump(vocab, f)

print(" Vocabulary saved successfully!")


 Vocabulary saved successfully!


In [ ]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [ ]:
# example of text tokenizing
vectorizer("Helloo everyone whassup my homies")[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([    1,   523,     1,    29, 37427])>

In [ ]:
# tokenizing the dataset
vectorizedText = vectorizer(x.values)

In [ ]:
vectorizedText

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [ ]:
tf.data.Dataset.list_files

<function tensorflow.python.data.ops.dataset_ops.DatasetV2.list_files(file_pattern, shuffle=None, seed=None, name=None) -> 'DatasetV2'>

In [ ]:
#MCSHBAP - map , cache ,shuffle ,batch ,prefetch from tenserflow slices
# batch the data
dataset = tf.data.Dataset.from_tensor_slices((vectorizedText,y))# passing the input data and the corresponding labels
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)# helps prevent bottlenecks
#Prefetching enables the system to load the next batch of data while the current batch is being processed by the GPU.


In [ ]:
dataset.as_numpy_iterator().next()
# load a batch

(array([[2149, 2149, 2149, ...,    0,    0,    0],
        [  19,    7,  988, ...,    0,    0,    0],
        [ 860,    4,  640, ...,    0,    0,    0],
        ...,
        [ 119,    3, 1626, ...,    0,    0,    0],
        [ 104,   76,  215, ...,    0,    0,    0],
        [  46,  168,   23, ...,    0,    0,    0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

In [ ]:
batch_x , batch_y = dataset.as_numpy_iterator().next()

In [ ]:
batch_x.shape # inputs
batch_y.shape

(16, 6)

In [ ]:
train = dataset.take(int(len(dataset )*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [ ]:
train_generator = train.as_numpy_iterator()

In [ ]:
train_generator.next()

(array([[    8,  5332,    22, ...,     0,     0,     0],
        [    7,    19,    15, ...,     0,     0,     0],
        [   21,    42,     6, ...,     0,     0,     0],
        ...,
        [   88,  1189, 98077, ...,     0,     0,     0],
        [  191,   288,     1, ...,     0,     0,     0],
        [47884, 45558, 50792, ...,     0,     0,     0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

create sequencial model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Dropout, Bidirectional,Dense ,Embedding

In [ ]:
model  = Sequential()# instantiate the sequential API so after that we can add the layers to the model?
# create the embedding layer

model.add(Embedding(MAX_WORDS+1,32)) # +1 is for the unknown words
# bidirectional LSTM layer
model.add(Bidirectional(LSTM(64 , activation = "tanh")))
# dense layers
model.add(Dense(128,activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(138, activation = "relu"))
model.add(Dropout(0.3))  # Drop 30% of neurons randomly. Dropout Layer (Before Final Output)

# converting the previous dense layer output to values between 0 and 1
model.add(Dense(6 , activation = "sigmoid"))
# dense layers connect each input to each output within its layer

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Compile the model with F1 score , accuracy, recall ,precision Metrics

In [ ]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * (precision * recall) / (precision + recall + epsilon())

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [ ]:
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.backend import epsilon


model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        Precision(name="precision"),
        Recall(name="recall"),
        AUC(name="auc"),
        F1Score()
    ]
)

MLOPs integrated training and validation

In [ ]:
# MLflow Experiment Setup
mlflow.set_experiment("toxicity_detection")

with mlflow.start_run():
    # Train Model with Logging
    history = model.fit(
        train,
        epochs=20,
        validation_data=val,
        callbacks=[
            EarlyStopping(monitor="val_loss", patience=3),
            WandbMetricsLogger(),  # Logs to W&B
            WandbModelCheckpoint(filepath="wandb_model.keras", save_best_only=True)  # ✅ Saves best model
        ]
    )

    # Retrieve Final Metrics
    final_accuracy = history.history["accuracy"][-1]
    final_loss = history.history["loss"][-1]

    # Log Metrics to MLflow
    mlflow.log_param("epochs", 20)
    mlflow.log_metric("accuracy", final_accuracy)
    mlflow.log_metric("loss", final_loss)

    # Save and Log Model
    model.save("toxicity_model.h5")
    mlflow.log_artifact("toxicity_model.h5")

    # Log Epoch-wise Metrics
    for epoch in range(len(history.history["loss"])):
        mlflow.log_metric("train_loss", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric("train_f1", history.history["f1_score"][epoch], step=epoch)
        mlflow.log_metric("val_f1", history.history["val_f1_score"][epoch], step=epoch)
        mlflow.log_metric("train_accuracy", history.history["accuracy"][epoch], step=epoch)
        mlflow.log_metric("val_accuracy", history.history["val_accuracy"][epoch], step=epoch)

    # Log Metrics to W&B
    wandb.log({
        "accuracy": final_accuracy,
        "loss": final_loss
    })

    # Plot Loss & F1-Score Curves
    def plot_and_log_metric(metric_name, title):
        plt.figure(figsize=(8, 5))
        plt.plot(history.history[metric_name], label="Train " + metric_name.capitalize())
        plt.plot(history.history["val_" + metric_name], label="Validation " + metric_name.capitalize())
        plt.xlabel("Epochs")
        plt.ylabel(metric_name.capitalize())
        plt.legend()
        plt.title(title)
        plt.savefig(f"{metric_name}_plot.png")
        mlflow.log_artifact(f"{metric_name}_plot.png")  # Log to MLflow
        wandb.log({f"{metric_name}_plot": wandb.Image(f"{metric_name}_plot.png")})  # Log to W&B
        plt.close()

    # Log Loss & F1-score Plots
    plot_and_log_metric("loss", "Loss over Epochs")
    plot_and_log_metric("f1_score", "F1 Score over Epochs")

    print(f"MLflow & W&B logging complete with accuracy: {final_accuracy:.2f}")

# Finish W&B Run
wandb.finish()

Epoch 1/20
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 733s 105ms/step - accuracy: 0.9910 - auc: 0.9849 - f1_score: 0.7567 - loss: 0.0447 - precision: 0.8301 - recall: 0.6953 - val_accuracy: 0.9937 - val_auc: 0.9913 - val_f1_score: 0.7823 - val_loss: 0.0396 - val_precision: 0.8483 - val_recall: 0.7259
Epoch 2/20
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 733s 105ms/step - accuracy: 0.9916 - auc: 0.9882 - f1_score: 0.7786 - loss: 0.0388 - precision: 0.8391 - recall: 0.7263 - val_accuracy: 0.9892 - val_auc: 0.9908 - val_f1_score: 0.8000 - val_loss: 0.0350 - val_precision: 0.8684 - val_recall: 0.7415
Epoch 3/20
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 733s 105ms/step - accuracy: 0.9933 - auc: 0.9912 - f1_score: 0.8024 - loss: 0.0349 - precision: 0.8509 - recall: 0.7592 - val_accuracy: 0.9946 - val_auc: 0.9935 - val_f1_score: 0.8105 - val_loss: 0.0310 - val_precision: 0.9038 - val_recall: 0.7347
Epoch 4/20
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 733s 105ms/step - accuracy: 0.9909 - auc: 0.9922 - f1_score: 0.8205 - loss: 0.0326 - pre

MLflow & W&B logging complete with accuracy: 0.93


accuracy,▁
epoch/accuracy,▇███████▇▄▄▇▇▇▇▇▁▄▅▇▆
epoch/auc,▁▆▆▇▇▇▇▇▇▇███████████
epoch/epoch,▁▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/f1_score,▁▃▄▄▅▅▆▆▆▇▇▇▇▇███████
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
epoch/precision,▁▃▃▄▄▄▅▅▆▆▆▆▇▇▇▇▇████
epoch/recall,▁▃▄▅▅▆▆▆▆▇▇▇▇▇███████
epoch/val_accuracy,█████████▁██████▇▃███
epoch/val_auc,▁▄▄▅▆▆▇▇▇▇▇▇█████████


In [ ]:
import wandb
import matplotlib.pyplot as plt
import mlflow

# Initialize W&B Run (Load previous run)
api = wandb.Api()
run = api.run("toxicity-detection/i2zqmk3c")
history = run.history()
print(history.columns)  # Shows all available metric names

# Convert W&B history to match your format
history_dict = {
    "loss": history["epoch/loss"].tolist(),
    "val_loss": history["epoch/val_loss"].tolist(),
    "f1_score": history["epoch/f1_score"].tolist(),
    "val_f1_score": history["epoch/val_f1_score"].tolist(),
    "accuracy": history["epoch/accuracy"].tolist(),
    "val_accuracy": history["epoch/val_accuracy"].tolist(),
    "precision": history["epoch/precision"].tolist(),
    "val_precision": history["epoch/val_precision"].tolist(),
    "recall": history["epoch/recall"].tolist(),
    "val_recall": history["epoch/val_recall"].tolist(),
    "auc": history["epoch/auc"].tolist(),
    "val_auc": history["epoch/val_auc"].tolist(),
    "learning_rate": history["epoch/learning_rate"].tolist(),  # No val version
}

# Define function to re-plot and log (handling missing val_ cases)
def plot_and_log_metric(metric_name, title, has_val=True):
    plt.figure(figsize=(8, 5))
    plt.plot(history_dict[metric_name], label="Train " + metric_name.capitalize())

    # Only plot validation if it exists
    if has_val:
        plt.plot(history_dict["val_" + metric_name], label="Validation " + metric_name.capitalize())

    plt.xlabel("Epochs")
    plt.ylabel(metric_name.capitalize())
    plt.legend()
    plt.title(title)
    plt.savefig(f"{metric_name}_plot.png")
    mlflow.log_artifact(f"{metric_name}_plot.png")  # Log to MLflow
    wandb.log({f"{metric_name}_plot": wandb.Image(f"{metric_name}_plot.png")})  # Log to W&B
    plt.close()

# Recreate and log all plots
plot_and_log_metric("loss", "Loss over Epochs")
plot_and_log_metric("f1_score", "F1 Score over Epochs")
plot_and_log_metric("accuracy", "Accuracy over Epochs")
plot_and_log_metric("precision", "Precision over Epochs")
plot_and_log_metric("recall", "Recall over Epochs")
plot_and_log_metric("auc", "AUC over Epochs")
plot_and_log_metric("learning_rate", "Learning Rate over Epochs", has_val=False)  # No validation version

print("Plots regenerated and logged successfully.")


Index(['loss', '_runtime', 'epoch/val_precision', 'epoch/accuracy',
       'epoch/f1_score', 'epoch/val_recall', 'accuracy', 'epoch/auc',
       'epoch/learning_rate', '_step', 'epoch/precision', 'epoch/loss',
       'epoch/recall', 'epoch/val_auc', 'loss_plot', 'epoch/val_f1_score',
       '_timestamp', 'epoch/val_accuracy', 'epoch/epoch', 'f1_score_plot',
       'epoch/val_loss'],
      dtype='object')
Plots regenerated and logged successfully.


In [ ]:
import mlflow
from tensorflow.keras.models import load_model  # Correct way to load a Keras model

# Load the pre-trained model
model = load_model("/content/toxicity_model.h5")

# End any active MLflow run if it exists
mlflow.end_run()

# If experiment does not exist, create one
experiment_name = "toxicity_detection"
try:
    experiment = mlflow.create_experiment(experiment_name)
except:
    # If experiment already exists, just get it
    experiment = mlflow.get_experiment_by_name(experiment_name)

# Start a new run within the experiment
with mlflow.start_run(experiment_id=experiment.experiment_id):
    # Log the model
    mlflow.keras.log_model(model, "model")

    # Assuming you have history_dict available (from your training history)
    # Log metrics to MLflow (no retraining required)
    for epoch in range(len(history_dict["loss"])):
        mlflow.log_metric("loss", history_dict["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history_dict["val_loss"][epoch], step=epoch)
        mlflow.log_metric("accuracy", history_dict["accuracy"][epoch], step=epoch)
        mlflow.log_metric("val_accuracy", history_dict["val_accuracy"][epoch], step=epoch)
        mlflow.log_metric("f1_score", history_dict["f1_score"][epoch], step=epoch)
        mlflow.log_metric("val_f1_score", history_dict["val_f1_score"][epoch], step=epoch)
        mlflow.log_metric("precision", history_dict["precision"][epoch], step=epoch)
        mlflow.log_metric("val_precision", history_dict["val_precision"][epoch], step=epoch)
        mlflow.log_metric("recall", history_dict["recall"][epoch], step=epoch)
        mlflow.log_metric("val_recall", history_dict["val_recall"][epoch], step=epoch)
        mlflow.log_metric("auc", history_dict["auc"][epoch], step=epoch)
        mlflow.log_metric("val_auc", history_dict["val_auc"][epoch], step=epoch)

    # Log the plots that were saved during training
    for metric_name in ["loss", "f1_score", "accuracy", "precision", "recall", "auc"]:
        mlflow.log_artifact(f"{metric_name}_plot.png")

print("Model and metrics have been logged successfully.")
import mlflow
import mlflow.keras

# Get experiment by name (or ID)
experiment = mlflow.get_experiment_by_name("toxicity_detection")

# Retrieve all runs
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])
print(runs.columns)

# Print the metrics for each run (e.g., accuracy, loss)
print(runs[["run_id", "metrics.accuracy", "metrics.loss"]])


2025/03/06 02:20:07 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/03/06 02:20:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model and metrics have been logged successfully.
Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.loss', 'metrics.auc', 'metrics.val_auc',
       'metrics.val_recall', 'metrics.accuracy', 'metrics.f1_score',
       'metrics.val_accuracy', 'metrics.val_loss', 'metrics.precision',
       'metrics.val_f1_score', 'metrics.val_precision', 'metrics.recall',
       'tags.mlflow.source.name', 'tags.mlflow.source.type',
       'tags.mlflow.runName', 'tags.mlflow.log-model.history',
       'tags.mlflow.user'],
      dtype='object')
                             run_id  metrics.accuracy  metrics.loss
0  fea7e2f8895642a793bf38a8a7362382               NaN           NaN
1  8942431403c44dada182b7fbf4fc3f40               NaN           NaN
2  0f1e7ada560a4f649a7ceea3e754f337               NaN           NaN
3  f3426f57943d41068963920ab123d900               NaN           NaN
4  a49a4770dcc14ec8b5f24786add8d9f0               NaN           NaN
5  86ac75d6

In [ ]:
import mlflow
import mlflow.keras

# Get experiment by name (or ID)
experiment = mlflow.get_experiment_by_name("toxicity_detection")

# Retrieve all runs
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])
print(runs.columns)

# Print the metrics for each run (e.g., accuracy, loss)
print(runs[["run_id", "metrics.accuracy", "metrics.loss"]])


Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.loss', 'metrics.auc', 'metrics.val_auc',
       'metrics.val_recall', 'metrics.accuracy', 'metrics.f1_score',
       'metrics.val_accuracy', 'metrics.val_loss', 'metrics.precision',
       'metrics.val_f1_score', 'metrics.val_precision', 'metrics.recall',
       'tags.mlflow.source.name', 'tags.mlflow.source.type',
       'tags.mlflow.runName', 'tags.mlflow.log-model.history',
       'tags.mlflow.user'],
      dtype='object')
                             run_id  metrics.accuracy  metrics.loss
0  a49a4770dcc14ec8b5f24786add8d9f0               NaN           NaN
1  86ac75d6a3f44eff836c0aee3efe2804               NaN           NaN


In [ ]:
from pyngrok import ngrok
!mlflow ui --port 5000 &
public_url = ngrok.connect(port="5000")
print(f"MLflow Tracking UI: {public_url}")

In [ ]:
from tensorflow.keras.layers import LSTM, Bidirectional

custom_objects = {"LSTM": LSTM, "Bidirectional": Bidirectional}

model = tf.keras.models.load_model('/content/toxicity_model.h5', custom_objects=custom_objects)


Make Predictions

In [ ]:
input_str = vectorizer("Heyy I hate you")
res = model.predict(np.expand_dims(input_str, axis = 0))
# printout the columns
print(df.columns[2:])
print(res)

input_str = vectorizer("You looks very nice")
res = model.predict(np.expand_dims(input_str, axis = 0))
# printout the columns
print(df.columns[2:])
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step
Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')
[[0.6889598  0.00483085 0.07420572 0.0524078  0.14637259 0.10724357]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')
[[5.3074697e-11 0.0000000e+00 2.5806218e-19 8.3492614e-38 8.7714351e-24
  1.4263595e-30]]


Evaluate Model on Test dataset

In [ ]:
batch_X, batch_y = test.as_numpy_iterator().next()
(model.predict(batch_X) > 0.5).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1]])

In [ ]:
res.shape

(1, 6)

In [ ]:
# Evaluate Model on Test Data
test_results = model.evaluate(test, verbose=1)

# Extract Metrics
test_loss = test_results[0]
test_accuracy = test_results[1]
test_precision = test_results[2]
test_recall = test_results[3]
test_auc = test_results[4]
test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall + tf.keras.backend.epsilon())

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test F1-Score: {test_f1:.4f}")
print(f"Test recall: {test_recall:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test loss: {test_loss:.4f}")


997/997 ━━━━━━━━━━━━━━━━━━━━ 180s 178ms/step - accuracy: 0.9877 - auc: 0.9975 - f1_score: 0.2112 - loss: 0.0091 - precision: 0.9523 - recall: 0.9477
Test Accuracy: 0.9866
Test F1-Score: 0.9498
Test recall: 0.9498
Test Precision: 0.9498
Test loss: 0.0095


In [ ]:
# Ensure W&B is initialized before logging
wandb.init(project="toxicity-detection", name="test_evaluation")

# Log Test Metrics to W&B
wandb.log({
    "test_loss": test_loss,
    "test_accuracy": test_accuracy,
    "test_precision": test_precision,
    "test_recall": test_recall,
    "test_f1_score": test_f1,
    "test_auc": test_auc
})

# Finish the W&B run
wandb.finish()


test_accuracy,▁
test_auc,▁
test_f1_score,▁
test_loss,▁
test_precision,▁
test_recall,▁
test_accuracy,0.98665
test_auc,0.99822
test_f1_score,0.94977
test_loss,0.00947
test_precision,0.94977


In [ ]:
!pip install fastapi uvicorn pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import tensorflow as tf
import numpy as np

app =FastAPI()

model =tf.keras.models.load_model("/content/toxicity_model.h5")
class InputText(BaseModel):
  text:str
@app.post("/predict")
async def predict(input_text : InputText):
  text: input_text.text
  vect_text = vectorizer(text)
  prediction = model.predict([text])
  return {"prediction": prediction.tolist()}

In [ ]:
from pyngrok import ngrok
!ngrok authtoken 2tVdIx30fQWA14bNtDn4lfBR9tE_77bGYGjjdXZRXD17uKHy5
!mlflow ui --port 5000 &
public_url = ngrok.connect(port="5000")
print(f"MLflow Tracking UI: {public_url}")

In [ ]:
from pyngrok import ngrok
!ngrok authtoken 2tVdIx30fQWA14bNtDn4lfBR9tE_77bGYGjjdXZRXD17uKHy5

!uvicorn --host 0.0.0.0 --port 8000 --reload app:app

public_url = ngrok.connect(8000)
print("FastAPI app is accessible at:", public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
FastAPI app is accessible at: NgrokTunnel: "https://5b11-34-59-186-82.ngrok-free.app" -> "http://localhost:8000"
INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [11838] using StatReload
ERROR:    Error loading ASGI app. Could not import module "app".
^C


In [ ]:
with mlflow.start_run():
    # Log Test Metrics
    mlflow.log_metric("test_loss", test_loss)
    mlflow.log_metric("test_accuracy", test_accuracy)
    mlflow.log_metric("test_precision", test_precision)
    mlflow.log_metric("test_recall", test_recall)
    mlflow.log_metric("test_auc", test_auc)
    mlflow.log_metric("test_f1_score", test_f1)

    print(" Test results logged in MLflow")


 Test results logged in MLflow


Test using Gradio UI

In [ ]:
from fastapi import FastAPI, WebSocket
import tensorflow as tf
import numpy as np
import json
import pickle
import os

#  Load the trained model
model = tf.keras.models.load_model("toxicity_model.h5")

#  Recreate the `TextVectorization` layer
MAX_WORDS = 200000
vectorizer = tf.keras.layers.TextVectorization(max_tokens=MAX_WORDS, output_sequence_length=1800, output_mode='int')

#  Define Toxicity Labels
TOXICITY_LABELS = ["Toxic", "Severe Toxic", "Obscene", "Threat", "Insult", "Identity Hate"]

#  Load the saved vocabulary
if os.path.exists("vectorizer_vocab.pkl"):
    with open("vectorizer_vocab.pkl", "rb") as f:
        vocab = pickle.load(f)

    if vocab:
        vectorizer.set_vocabulary(vocab)
        print(" Vocabulary loaded successfully!")
    else:
        print("🚨 Error: Loaded vocabulary is empty!")
        exit(1)
else:
    print("🚨 Error: vectorizer_vocab.pkl not found!")
    exit(1)

#  Ensure Model is Compiled
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#  Store connected users
connected_users = set()

#  Function to Predict Toxicity & Return Scores
def predict_toxicity(message):
    if not message.strip():
        return [0.0] * 6

    input_text = tf.expand_dims(message, axis=0)
    input_text = vectorizer(input_text)

    prediction = model.predict(input_text)[0]
    toxicity_scores = {label : round(float(score),4) for label,score in zip(TOXICITY_LABELS, prediction)}
    return toxicity_scores  # Returns list of scores

#  Start FastAPI
app = FastAPI()

@app.websocket("/chat")
async def chatroom(websocket: WebSocket):
    await websocket.accept()
    connected_users.add(websocket)

    try:
        while True:
            #  Receive message from user
            data = await websocket.receive_text()
            message_data = json.loads(data)
            message = message_data["message"]
            user = message_data["user"]

            #  Predict toxicity & get max score
            toxicity_scores = predict_toxicity(message)
            max_toxicity = max(toxicity_scores.values())

            # display color according to toxic level
            def format_toxicity_scores(toxicity_scores):
                return "\n".join(
                    f"{'🔴' if score >= 0.7 else '🟢'} {category}: {score * 100:.2f}%"
                    for category, score in toxicity_scores.items()
                )

            if max_toxicity > 0.7:
                #  Send warning **only to the sender** with full toxicity details
                formatted_scores = format_toxicity_scores(toxicity_scores)
                await websocket.send_json({
                    "user": "assistant",
                    "message": "🚨 Message Blocked Due to High Toxicity!" + formatted_scores
                })
                continue  # Don't broadcast toxic messages

            #  Broadcast Clean Messages + Toxicity Score to Everyone
            response_message = {
                "user": user,
                "message": message,
                "toxicity_score": max_toxicity  # Include toxicity score
            }
            for user_socket in connected_users:
                await user_socket.send_json(response_message)

    except Exception as e:
        print(f"Error: {e}")
    finally:
        connected_users.remove(websocket)  # Remove disconnected users

#  Run FastAPI Server Locally
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="localhost", port=8000)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.


In [ ]:
pip install pyngrok

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok gradio tensorflow websockets


In [ ]:
!ngrok authtoken XUV4QONYGOEHMJ26ZWV23SI6E53S3FXK

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
